In [1]:
import streamlit as st
import gspread as gs 

SPREADSHEET = 'portfolio'

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets[SPREADSHEET].spreadsheet_key).worksheet(st.secrets[SPREADSHEET].worksheet_name)

In [2]:
import yfinance as yf

eurusd = yf.download('EURUSD=X', start='2021-01-01')[['Close']].reset_index()

[*********************100%***********************]  1 of 1 completed


In [3]:
import pandas as pd

df = pd.DataFrame(ss.get_all_records())
df.query('Portfolio == "ZEN REMIX"', inplace=True)
df['EURUSD'] = df.apply(lambda row: eurusd.query("Date == @row['Date']")['Close'].values[0], axis=1)
df['Total€'] = df.apply(lambda row: row['Total'] / (1 if row['Currency'] == "€" else row['EURUSD']), axis=1)
df['Value€'] = df.apply(lambda row: row['Total'] / (1 if row['Currency'] == "€" else row['EURUSD']), axis=1)
df.head()


,Date,Portfolio,Account,Asset,Price,Amount,Fees,Total,Total€,Currency,Out,Profit,EURUSD
0,30 Apr 2021,ZEN REMIX,eToro,$,,,,598.0,493.110802,$,,,1.212709
1,30 Apr 2021,ZEN REMIX,eToro,GREENBULL,598,1,,598.0,493.110802,$,,,1.212709
2,30 Jul 2021,ZEN REMIX,eToro,$,,,,585.0,491.886106,$,,,1.189300
3,30 Jul 2021,ZEN REMIX,eToro,GREENBULL,585,1,,585.0,491.886106,$,,,1.189300
5,10 Aug 2021,ZEN REMIX,B.Direct CTO,€,,,,300.0,300.000000,€,,,1.173847


In [4]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Total€']))
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed